# 2.assess-heterogeneity 

This section of the notebook uses buscar's clustering module to assess single-cell heterogeneity. We'll focus on three specific datasets: **CFReT**, **MitoCheck**, and **CPJUMP (crispir)**. The goal is to use our clustering algorithms to identify cellular heterogeneity at the single-cell level. 

A key advantage of using these datasets is that they include ground-truth labels. This allows us to evaluate whether our clustering algorithms are identifying biologically meaningful groups in a data-driven way, and to assess the accuracy of our approach.

In [3]:
import sys
import pathlib


sys.path.append("../../")
from utils.heterogeneity import optimized_clustering, cluster_profiles
from utils.io_utils import load_profiles

Setting parameter grid search for the optimized clustering 

In [4]:
cluster_search_param_grid = {
    # Clustering resolution: how granular the clusters should be
    "cluster_resolution": {"type": "float", "low": 0.1, "high": 2.0},
    # Number of neighbors for graph construction
    "n_neighbors": {"type": "int", "low": 5, "high": 99},
    # Clustering algorithm
    "cluster_method": {"type": "categorical", "choices": ["leiden", "louvain"]},
    # Distance metric for neighbor computation
    "neighbor_distance_metric": {
        "type": "categorical",
        "choices": ["euclidean", "cosine", "manhattan"],
    },
    # Dimensionality reduction approach
    "dim_reduction": {"type": "categorical", "choices": ["PCA", "raw"]},
}

Setting paths

In [7]:
# set module and data directory paths
download_module_path = pathlib.Path("../0.download-data/").resolve(strict=True)
sc_profiles_path = (download_module_path / "data" / "sc-profiles").resolve(strict=True)


# setting profiles paths
cfret_profiles_path = (
    sc_profiles_path / "cfret" / "localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)
cpjump1_trt_crispr_profiles_path = (
    sc_profiles_path
    / "cpjump1"
    / "trt-profiles"
    / "cpjump1_crispr_trt_profiles.parquet"
).resolve(strict=True)
mitocheck_trt_profiles_path = (
    sc_profiles_path / "mitocheck" / "mitocheck_concat_profiles.parquet"
).resolve(strict=True)

# create signature output paths
results_dir = pathlib.Path("./results/cluster-labels").resolve()
results_dir.mkdir(exist_ok=True, parents=True)

Loading datasets

In [8]:
# load all profiles
# mitocheck_profiles_df = load_profiles(mitocheck_trt_profiles_path)
cfret_profiles_df = load_profiles(cfret_profiles_path)
# cpjump1_crispr_profiles_df = load_profiles(cpjump1_trt_crispr_profiles_path)

## Clustering profiles

### Assessing heterogeneity for MitoCheck data

In [5]:
# separate metadata based on phenotypic class
# split metadata and features
mito_meta = [
    "index",
    "Mitocheck_Phenotypic_Class",
    "Cell_UUID",
    "Metadata_treatment_type",
    "Metadata_cell_id",
    "Location_Center_X",
    "Location_Center_Y",
    "Metadata_Plate",
    "Metadata_Well",
    "Metadata_Frame",
    "Metadata_Site",
    "Metadata_Plate_Map_Name",
    "Metadata_DNA",
    "Metadata_Gene",
    "Metadata_Gene_Replicate",
]

mito_features = mitocheck_profiles_df.drop(mito_meta).columns

In [ ]:
# does not work getting an error about dealing with sparse matrices
cluster_profiles(
    profiles=mitocheck_profiles_df,
    meta_features=mito_meta,
    morph_features=mito_features,
    treatment_col="Metadata_Gene",
    pca_n_components_to_capture_variance=100,
)

### Assessing heterogeneity for CFReT data

In [9]:
# split metadata and features for cfret
cfret_meta = [
    "Metadata_WellRow",
    "Metadata_WellCol",
    "Metadata_heart_number",
    "Metadata_cell_type",
    "Metadata_heart_failure_type",
    "Metadata_treatment",
    "Metadata_Nuclei_Location_Center_X",
    "Metadata_Nuclei_Location_Center_Y",
    "Metadata_Cells_Location_Center_X",
    "Metadata_Cells_Location_Center_Y",
    "Metadata_Image_Count_Cells",
    "Metadata_ImageNumber",
    "Metadata_Plate",
    "Metadata_Well",
    "Metadata_Cells_Number_Object_Number",
    "Metadata_Cytoplasm_Parent_Cells",
    "Metadata_Cytoplasm_Parent_Nuclei",
    "Metadata_Nuclei_Number_Object_Number",
    "Metadata_Site",
]

cfret_feats = cfret_profiles_df.drop(cfret_meta).columns

In [10]:
cfret_cluster_search_param_grid = {
    # Clustering resolution: how granular the clusters should be
    "cluster_resolution": {"type": "float", "low": 0.1, "high": 2.2},
    # Number of neighbors for graph construction
    "n_neighbors": {"type": "int", "low": 5, "high": 100},
    # Clustering algorithm
    "cluster_method": {"type": "categorical", "choices": ["leiden", "louvain"]},
    # Distance metric for neighbor computation
    "neighbor_distance_metric": {
        "type": "categorical",
        "choices": ["euclidean", "cosine", "manhattan"],
    },
    # Dimensionality reduction approach
    "dim_reduction": {"type": "categorical", "choices": ["PCA", "raw"]},
}

In [11]:
cfret_cluster_results, cfret_best_params = optimized_clustering(
    profiles=cfret_profiles_df,
    morph_features=cfret_feats,
    meta_features=cfret_meta,
    treatment_col="Metadata_treatment",
    n_trials=500,
    param_grid=cluster_search_param_grid,
    n_jobs=20,
)

[I 2025-10-10 10:00:13,501] A new study created in memory with name: cluster_optimization_0
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/numba/np/ufunc/dufunc.py:346: NumbaWarning: Compilation requested for previously compiled argument types ((float32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/erikserrano/So

In [8]:
cfret_best_params

{'cluster_resolution': 1.8478343159603974,
 'n_neighbors': 47,
 'cluster_method': 'leiden',
 'neighbor_distance_metric': 'euclidean',
 'dim_reduction': 'PCA'}

In [9]:
cfret_cluster_results

Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Site,Metadata_cell_id,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_4_2,…,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_01_256,Nuclei_Texture_InfoMeas1_ER_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_Hoechst_3_00_256,Nuclei_Texture_InfoMeas1_Hoechst_3_01_256,Nuclei_Texture_InfoMeas1_Hoechst_3_02_256,Nuclei_Texture_InfoMeas1_Hoechst_3_03_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_00_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_01_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_02_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_03_256,Nuclei_Texture_InfoMeas1_PM_3_00_256,Nuclei_Texture_InfoMeas1_PM_3_01_256,Nuclei_Texture_InfoMeas1_PM_3_02_256,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_InfoMeas2_ER_3_01_256,Nuclei_Texture_InfoMeas2_ER_3_03_256,Nuclei_Texture_InfoMeas2_Hoechst_3_01_256,Nuclei_Texture_InfoMeas2_Hoechst_3_03_256,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256,Metadata_cluster_id,Metadata_cluster_n_cells,Metadata_treatment_n_cells,Metadata_cluster_ratio
str,i64,i64,str,str,str,f64,f64,f64,f64,i64,i64,str,str,i64,i64,i64,i64,str,u64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,cat,u32,u32,f64
"""B""",2,9,"""failing""","""rejected""","""DMSO""",221.046761,137.115493,246.6028,109.285755,40,1,"""localhost230405150001""","""B02""",1,1,6,6,"""f00""",3741745246391810767,-1.35494,0.841229,0.648883,-0.850138,-1.045214,1.298358,0.376165,0.935101,1.530228,-0.983617,-0.261031,-0.299817,-0.721977,0.944725,0.161074,0.532329,1.845864,…,-0.173336,0.300041,0.217945,-0.039774,0.488531,0.472164,0.28659,0.464359,0.501649,0.507623,1.076663,0.741941,-0.696022,-0.178762,0.186741,0.158222,0.341595,0.50487,-0.440604,-0.426966,0.194372,-0.035117,0.400021,-0.619206,-0.393448,0.961214,0.406068,0.374039,-0.280532,-0.158967,-0.344804,-0.263653,-0.305486,"""DMSO_leiden_45""",127,10519,1.207339
"""B""",2,9,"""failing""","""rejected""","""DMSO""",690.596142,183.067828,716.170091,177.132195,40,1,"""localhost230405150001""","""B02""",2,2,7,7,"""f00""",2742388086301918299,0.657107,-0.850399,-0.584931,2.090925,1.263259,-0.021031,1.627957,0.944161,-0.085511,1.475345,2.164761,-0.688462,1.215015,1.499086,-0.770667,1.012721,0.6791,…,0.134835,0.263514,-0.124309,0.634517,0.968512,0.859562,0.351144,0.914468,-2.508508,-2.389124,-1.80698,-2.121536,-0.23123

### Assessing heterogeneity for CPJUMP1 CRISPR data

In [14]:
# split metadata and features for cpjump1
cpjump1_meta = [
    "index",
    "Metadata_broad_sample",
    "Metadata_ImageNumber",
    "Metadata_Plate",
    "Metadata_Site",
    "Metadata_Well",
    "Metadata_TableNumber",
    "Metadata_ObjectNumber_cytoplasm",
    "Metadata_Cytoplasm_Parent_Cells",
    "Metadata_Cytoplasm_Parent_Nuclei",
    "Metadata_ObjectNumber_cells",
    "Metadata_ObjectNumber",
    "Metadata_gene",
    "Metadata_pert_type",
    "Metadata_control_type",
    "Metadata_target_sequence",
    "Metadata_negcon_control_type",
    "__index_level_0__",
]

# split metadata and features for cpjump1
cpjump1_feats = cpjump1_crispr_profiles_df.drop(cpjump1_meta).columns

In [17]:
cpjump1_cluster_results = optimized_clustering(
    profiles=cpjump1_crispr_profiles_df,
    meta_features=cpjump1_meta,
    morph_features=cpjump1_feats,
    treatment_col="Metadata_gene",
    param_grid=cluster_search_param_grid,
    n_trials=30,
)

[I 2025-10-06 09:36:13,825] A new study created in memory with name: cluster_optimization_0
[I 2025-10-06 10:54:32,232] Trial 0 finished with value: 0.21206753315840984 and parameters: {'cluster_resolution': 1.6915591613892418, 'n_neighbors': 37, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'cosine', 'dim_reduction': 'raw'}. Best is trial 0 with value: 0.21206753315840984.
[I 2025-10-06 12:15:45,314] Trial 1 finished with value: 0.2291905488765506 and parameters: {'cluster_resolution': 1.2119804045947553, 'n_neighbors': 41, 'cluster_method': 'louvain', 'neighbor_distance_metric': 'euclidean', 'dim_reduction': 'raw'}. Best is trial 1 with value: 0.2291905488765506.
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(
[I 2025-10-06 13:13:59,118] Trial 2 finished with valu

: 